<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
from IPython.display import display


# Загрузка CSV файла в DataFrame
df = pd.read_csv("Отчет по текущим тендерам.csv")



# Разделение по номеру строки
df1 = df.loc[0:len(df)//2]  # Первая половина по номерам строк
df2 = df.loc[len(df)//2 + 1:]  # Вторая половина по номерам строк


# Проверка
display(df1)
display(df2)


about_company = '...'


def send_data(data):
    return '...'


def analyze(data):
    response = send_data(data)
    result = about_company == response
    data['analyzed'] = result
    return data



def convert_to_dict(row):
  """
  Преобразует строку в словарь,
  используя значения столбцов в качестве ключей.
  """
  return {col: str(val).strip('"').strip("'") for col, val in zip(df.columns, row)}


def process(row):
    data = convert_to_dict(row)
    new_dict = analyze(data)
    return new_dict


df1_dicts = df1.apply(process, axis=1)


print(df1_dicts[0])
print(df1_dicts[1])

,Дата внесения в таблицу,Объявление №,Лот №,Наименование лота,Способ закупок,Статус закупки,Дата начала,Дата окончания,Начальная максимальна цена,Кол-во,...,Руководитель,Менеджер,Решение,Комментарий,Допуск,Победитель,Цена,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,17.03.2023,2036727,NaN,КРУН 10 кВ Западная,NaN,Завершено,NaN,29.03.2023,"290 000 000,00",,...,Манатов А.Б.,Берик,участвуем,NaN,Допущен,"ТОО ""АЭТЗ""",NaN,NaN,NaN,NaN
1,17.03.2023,9499432-2,NaN,Устройства распределительные,NaN,Завершено,NaN,31.03.2023,"163 660 117,00",,...,Манатов А.Б.,Берик,участвуем,"Признана несостоявшейся в связи с тем, что к у...",Отклонен,Не состоялся,NaN,NaN,NaN,NaN
2,18.03.2023,2036981,NaN,КТП,NaN,Завершено,NaN,04.04.2023,"102 093 121,00",,...,Манатов А.Б.,Берик,участвуем,прошу отправить ОЛ либо проект для ознакомлени...,не участвовали,Протокол не доступен,NaN,NaN,NaN,NaN
3,20.03.2023,000010363 - eurasiantech,NaN,КТП,NaN,Завершено,NaN,03.04.2023,"30 322 563,00",,...,Манатов А.Б.,Берик,участвуем,прошу отправить ОЛ либо проект для ознакомлени...,Допущен,ТОО Bolashak Electric,NaN,NaN,NaN,NaN
4,20.03.2023,10460-eurasiantech,NaN,КСО-3,NaN,Завершено,NaN,05.04.2023,"2 198 600,00",,...,Манатов А.Б.,NaN,участвуем,"програли, не прошли по цене",Отклонен,ТОО ДИКОМ ЭНЕРДЖИ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,25.05.2023,EE15038,2023-15038,Трансформатор ТМ-4000/35-6кВ с учетом шеф-мон...,Тендер,Завершено,24.05.2023,07.06.2023,40 263 872.32,1,...,Манатов А.Б.,Сергей,Не участвуем,отгрузка по заказу покупателя в течение 60 кал...,NaN,NaN,NaN,NaN,NaN,NaN
209,25.05.2023,EE15038,2023-15038,Трансформатор ТМН-6300/35-10кВ с учетом шеф-мо...,Тендер,Завершено,24.05.2023,07.06.2023,61 478 545.54,1,...,Манатов А.Б.,Сергей,Не участвуем,отгрузка по заказу покупателя в течение 60 кал...,NaN,NaN,NaN,NaN,NaN,NaN
210,25.05.2023,EE15038,2023-15038,"Трансформатор ТМ-6300/35/10кВ, с учетом шеф-м...",Тендер,Завершено,24.05.2023,07.06.2023,49 460 709.82,1,...,Манатов А.Б.,Сергей,Не участвуем,отгрузка по заказу покупателя в течение 60 кал...,NaN,NaN,NaN,NaN,NaN,NaN
211,25.05.2023,EE15038,2023-15038,"Трансформатор ТМ-6300/35/6кВ, с учетом шеф-мон...",Тендер,Завершено,24.05.2023,07.06.2023,49 460 709.82,1,...,Манатов А.Б.,Сергей,Не участвуем,отгрузка по заказу покупателя в течение 60 кал...,NaN,NaN,NaN,NaN,NaN,NaN


,Дата внесения в таблицу,Объявление №,Лот №,Наименование лота,Способ закупок,Статус закупки,Дата начала,Дата окончания,Начальная максимальна цена,Кол-во,...,Руководитель,Менеджер,Решение,Комментарий,Допуск,Победитель,Цена,Unnamed: 24,Unnamed: 25,Unnamed: 26
213,25.05.2023,№ 000012672,NaN,Шкаф оперативного тока ШОТ,Конкурс,Завершено,24.05.2023,07.06.2023,"28 283 000,00",1,...,Манатов А.Б.,Сергей,Не участвуем,тендер под производителя,NaN,NaN,NaN,NaN,NaN,NaN
214,26.05.2023,G10003494-1,61433230-ОК1,Работы по строительству/монтажу/прокладке сете...,Конкурс посредством электронных закупок,Дополнение заявок,29.05.2023,13.06.2023,585 882 360.00,1,...,Сисенов Э.Ж.,NaN,не участвуем,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,26.05.2023,G9996859-1,61430952-ОК1,Работы по строительству/монтажу/прокладке сете...,Конкурс посредством электронных закупок,Дополнение заявок,29.05.2023,13.06.2023,245 833 401.00,1,...,Сисенов Э.Ж.,NaN,не участвуем,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,26.05.2023,Z862862,3223966,Трансформатор напряжения,Тендер,Завершено,22.05.2023,26.05.2023,8 128 446.00,1,...,Манатов А.Б.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,26.05.2023,Z862862,3223965,Трансформатор напряжения,Тендер,Завершено,22.05.2023,26.05.2023,44 029 086.00,3,...,Манатов А.Б.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,04.12.2023,11105415-1,64217195-ОК1,\nПроведение электромонтажных и пусконаладочны...,открый,обсуждение,07.12.2023,23.12.2023,"15 046 418,75",1,...,Сисенов Э.,Сергей,NaN,Задача создана,NaN,NaN,NaN,NaN,NaN,NaN
421,04.12.2023,11105415-1,\t64244193-ОК1,Проведение электромонтажных и пусконаладочных ...,открый,обсуждение,07.12.2023,23.12.2023,"64 765 529,47",1,...,Сисенов Э.,Сергей,NaN,Задача создана,NaN,NaN,NaN,NaN,NaN,NaN
422,04.12.2023,11071171-1,1,\nСтроительно-монтажные работы «Электроснабжен...,открый,прием заявок,01.12.2023,16.12.2023,"80 048 158,00",1,...,Сисенов Э.,Кабдуллин Р,NaN,Задача создана,NaN,NaN,NaN,NaN,NaN,NaN
423,07.12.2023,11126053-1,\t63469664-ЗЦП4,комплект для модернизации шкаф КРУ K-XII/ комп...,Запрос ценовых предложений,прием заявок,06.12.2023,13.12.2023,"6 696 429,00",3,...,Сисенов Э.,Ниязов Д.,NaN,Задача создана,NaN,NaN,NaN,NaN,NaN,NaN


{'Дата внесения в таблицу': '17.03.2023', 'Объявление №': '2036727', 'Лот №': 'nan', 'Наименование лота': 'КРУН 10 кВ Западная', 'Способ закупок': 'nan', 'Статус закупки ': 'Завершено ', 'Дата начала': 'nan', 'Дата окончания': '29.03.2023', 'Начальная максимальна цена ': '290\xa0000\xa0000,00', 'Кол-во': '\xa0', 'Ед.изм.': '\xa0', 'Сумма': 'nan', 'Заказчик': 'ТОО Кокшетау-Энерго', 'Регион': 'Кокшетау', 'Источник': 'https://www.ets-tender.kz/market/view.html?id=2036727', 'Контакты заказчика': 'nan', 'Тендерный специалист ': 'Приходько', 'Руководитель ': 'Манатов А.Б.', 'Менеджер ': 'Берик', 'Решение': 'участвуем', 'Комментарий': 'nan', 'Допуск ': 'Допущен', 'Победитель': 'ТОО "АЭТЗ', 'Цена': 'nan', 'Unnamed: 24': 'nan', 'Unnamed: 25': 'nan', 'Unnamed: 26': 'nan', 'analyzed': True}
{'Дата внесения в таблицу': '17.03.2023', 'Объявление №': '9499432-2', 'Лот №': 'nan', 'Наименование лота': 'Устройства распределительные', 'Способ закупок': 'nan', 'Статус закупки ': 'Завершено ', 'Дата начал